# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [41]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'#'analogy_alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [42]:

import codecs, scipy.sparse, numpy

#create co-occurrence matrix
def create_cooccurrence_matrix(filename,window_size):
    vocabulary={}
    data=[]
    row=[]
    col=[]
    for sentence in codecs.open(filename,"r","utf-8"):
        sentence=sentence.strip()
        tokens=[token for token in corpus]
        for pos,token in enumerate(tokens):
            i=vocabulary.setdefault(token,len(vocabulary))
            start=max(0,pos-window_size_corpus)
            end=min(len(tokens),pos+window_size_corpus+1)
            for pos2 in xrange(start,end):
                if pos2==pos: 
                    continue
                j=vocabulary.setdefault(tokens[pos2],len(vocabulary))
                data.append(1.); row.append(i); col.append(j);
    cooccurrence_matrix=sparse.coo_matrix((data,(row,col)))
    return vocabulary,cooccurrence_matrix

#create_cooccurrence_matrix('alice.txt',corpus,window_size)
#print(corpus)
#coo=numpy.zeros((V, V))
#coo=[[0 for x in range(V)] for y in range(V)] 
vocabulary={}
data=[]
row=[]
col=[]
for sentence in corpus:
    for pos,word in enumerate(sentence):
        i=vocabulary.setdefault(word,len(vocabulary))
        start=max(0,pos-window_size_corpus)
        end=min(len(sentence),pos+window_size_corpus+1)
        for pos2 in range(start,end):
            if pos2==pos: 
                continue
            j=vocabulary.setdefault(sentence[pos2],len(vocabulary))
            if(j!=i):
                data.append(1.); row.append(i); col.append(j);
            
cooccurrence_matrix=scipy.sparse.coo_matrix((data,(row,col)))
#print(cooccurrence_matrix.T * cooccurrence_matrix)
print(cooccurrence_matrix.toarray())
#print(cooccurrence_matrix.toarray()[1180][0])
tokenizer.word_index
#The dog chased the cat away from the garden

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 8. ... 0. 0. 0.]
 [1. 8. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


{'the': 1,
 'and': 2,
 'to': 3,
 'she': 4,
 'a': 5,
 'i': 6,
 'it': 7,
 'of': 8,
 'was': 9,
 'in': 10,
 'alice': 11,
 'you': 12,
 'that': 13,
 'her': 14,
 'as': 15,
 'said': 16,
 'had': 17,
 'for': 18,
 'but': 19,
 'be': 20,
 'on': 21,
 'all': 22,
 'with': 23,
 'little': 24,
 'mouse': 25,
 'down': 26,
 'very': 27,
 'this': 28,
 'not': 29,
 'so': 30,
 'out': 31,
 'if': 32,
 'is': 33,
 'at': 34,
 't': 35,
 's': 36,
 'll': 37,
 'how': 38,
 'they': 39,
 'about': 40,
 'herself': 41,
 'me': 42,
 'up': 43,
 'what': 44,
 'way': 45,
 'when': 46,
 'like': 47,
 'one': 48,
 'do': 49,
 'no': 50,
 'oh': 51,
 'went': 52,
 'thought': 53,
 'again': 54,
 'there': 55,
 'see': 56,
 'or': 57,
 'could': 58,
 'would': 59,
 'think': 60,
 'them': 61,
 'know': 62,
 'rabbit': 63,
 'dear': 64,
 'were': 65,
 'time': 66,
 'get': 67,
 'here': 68,
 'must': 69,
 'my': 70,
 'by': 71,
 'into': 72,
 'found': 73,
 'such': 74,
 'began': 75,
 'soon': 76,
 'm': 77,
 'quite': 78,
 'then': 79,
 'off': 80,
 'now': 81,
 'go': 82

In [43]:
#find cosine similarity to Alice, Dinah and Rabbit
#print(corpus)

#from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf_vectorizer = TfidfVectorizer()
#tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
#print(tfidf_matrix.shape)
#from gensim.models import TfidfModel


print(tokenizer.word_index['alice'])  #11
print(tokenizer.word_index['dinah'])  #87
print(tokenizer.word_index['rabbit']) #63



cooc=cooccurrence_matrix.toarray()
tf_cooc=numpy.zeros((V, V))
for i, sentence in enumerate(cooc):
    sum=0
    for wordf in sentence:
        sum+=wordf
    for j,wordf in enumerate(sentence):
        if(sum>0):
            tf_cooc[i][j]=cooc[i][j]/sum
        else:
            continue
#print(tf_cooc)

#print(len(cooc))
#print(len(cooc[10]))
#print(V)
#print(cooc[0])
#print(tokenizer.word_index)

from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(tf_cooc[10:11], tf_cooc[86:87]))
print(cosine_similarity(tf_cooc[10:11], tf_cooc[62:63]))
print(cosine_similarity(tf_cooc[86:87], tf_cooc[62:63]))

11
87
63
[[0.48967328]]
[[0.08862668]]
[[0.13969969]]


In [ ]:
#find the closest words to Alice


Discussion of the drawbacks:

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [ ]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow


In [242]:
#create CBOW model


In [243]:
#define loss function


In [1]:
#train model


In [23]:
#prepare data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    in_words.append([word])
                    labels.append(words[i])
            if in_words != []:
                all_in.append(np.array(in_words,dtype=np.int32))
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out)

#get x and y's for data
x,y = generate_data_skipgram(corpus,window_size,V)

#save the preprocessed data of Skipgram
f = open('data_skipgram.txt' ,'w')

for input,outcome  in zip(x,y):
    input = np.concatenate(input)
    f.write(" ".join(map(str, list(input))))
    f.write(",")
    outcome = np.concatenate(outcome)
    f.write(" ".join(map(str,list(outcome))))
    f.write("\n")
f.close()

#load the preprocessed Skipgram data
def generate_data_skipgram_from_file():
    f = open('data_skipgram.txt' ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

In [24]:
#create Skipgram model
skipgram = Sequential()
skipgram.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

In [25]:
#define loss function for Skipgram
skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [26]:
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram.txt' ,'w')
f.write(" ".join([str(V-1),str(dim)]))
f.write("\n")
vectors = skipgram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

0 42011.7700715065
1 38353.8222258091
2 38931.23595356941
3 39340.806143045425
4 39514.309755563736
5 39681.91601896286
6 39847.13162755966
7 40027.34559297562
8 40214.910954236984
9 40403.25219488144


In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [27]:
#create Skipgram with additional dense layer
skipgram2 = Sequential()
skipgram2.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram2.add(Reshape((dim, )))
skipgram2.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
skipgram2.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

In [28]:
#define loss function for Skipgram + dense
skipgram2.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [29]:
#train model for Skipgram + dense
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram2.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram2.txt' ,'w')
f.write(" ".join([str(V-1),str(dim)]))
f.write("\n")
vectors = skipgram2.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

0 41944.31506729126
1 38392.68016695976
2 39111.658846616745
3 39808.2161591053
4 40187.521782159805
5 40399.22357773781
6 40525.47047138214
7 40603.02894616127
8 40627.425698041916
9 40142.643817186356


In [31]:
#Implement your own analogy function


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [7]:
#load pretrained word embeddings of word2vec

path_word2vec = "tes\GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path_word2vec, binary=True)

In [21]:
#load pretraind word embeddings of Glove
import gensim
#path = "tes\glove.6B\glove.6B.300d_converted.txt"
path = "tes\glove.6B.300d.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

ValueError: invalid literal for int() with base 10: 'the'

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings

In [ ]:
tokenizer.word_index